In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
13,application_1632897281066_0014,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
salesDF = glueContext.create_dynamic_frame.from_catalog(
             database="johndb",
             table_name="sales")
customerDF = glueContext.create_dynamic_frame.from_catalog(
             database="johndb",
             table_name="customers")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
salesDF.printSchema()
customerDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- ordernumber: long
|-- quantityordered: long
|-- priceeach: double
|-- orderlinenumber: long
|-- sales: double
|-- orderdate: string
|-- status: string
|-- qtr_id: long
|-- month_id: long
|-- year_id: long
|-- productline: string
|-- msrp: long
|-- productcode: string
|-- dealsize: string
|-- customerid: long

root
|-- customerid: long
|-- customername: string
|-- email: string
|-- city: string
|-- country: string
|-- territory: string
|-- contactfirstname: string
|-- contactlastname: string

In [4]:
customersalesDF=Join.apply(salesDF, customerDF, 'customerid', 'customerid')
customersalesDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- orderlinenumber: long
|-- sales: double
|-- territory: string
|-- city: string
|-- customername: string
|-- year_id: long
|-- productline: string
|-- contactlastname: string
|-- country: string
|-- .customerid: long
|-- qtr_id: long
|-- msrp: long
|-- quantityordered: long
|-- productcode: string
|-- status: string
|-- email: string
|-- contactfirstname: string
|-- ordernumber: long
|-- orderdate: string
|-- customerid: long
|-- dealsize: string
|-- month_id: long
|-- priceeach: double

In [5]:
customersalesDF = customersalesDF.drop_fields(['`.customerid`'])
customersalesDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- orderlinenumber: long
|-- sales: double
|-- territory: string
|-- city: string
|-- customername: string
|-- year_id: long
|-- productline: string
|-- contactlastname: string
|-- country: string
|-- qtr_id: long
|-- msrp: long
|-- quantityordered: long
|-- productcode: string
|-- status: string
|-- email: string
|-- contactfirstname: string
|-- ordernumber: long
|-- orderdate: string
|-- customerid: long
|-- dealsize: string
|-- month_id: long
|-- priceeach: double

In [3]:
#You can split a Dynamicframe vertically based on the columns. Use SplitFields method which splits a Dynamicframe into a collection of Dynamicframes. 
colwiseCollection = SplitFields.apply(salesDF,["productline","dealsize","status"],"productsDF","restDF")
colwiseCollection.keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['productsDF', 'restDF'])

In [4]:
colwiseCollection.select("productsDF").printSchema()
colwiseCollection.select("restDF").printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- productline: string
|-- dealsize: string
|-- status: string

root
|-- ordernumber: long
|-- quantityordered: long
|-- priceeach: double
|-- orderlinenumber: long
|-- sales: double
|-- orderdate: string
|-- qtr_id: long
|-- month_id: long
|-- year_id: long
|-- msrp: long
|-- productcode: string
|-- customerid: long

In [5]:
# you can split a Dynamicframe horizontally based on the row. Use SplitRows method which splits a Dynamicframe into a collection of Dynamicframes based on the condition specified for the rows. 
rowwiseCollection = SplitRows.apply(salesDF,{"sales": {">": 5000.00}},"sales5000plus","sales5000minus")
rowwiseCollection.keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['sales5000plus', 'sales5000minus'])

In [6]:
rowwiseCollection.select("sales5000minus").toDF().select('sales').show()
rowwiseCollection.select("sales5000plus").toDF().select('sales').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|  sales|
+-------+
| 2871.0|
| 2765.9|
|3884.34|
| 3746.7|
|3479.76|
|2497.77|
|2168.54|
|4708.44|
|3965.66|
|2333.12|
|3188.64|
|3676.76|
|4177.35|
|4099.68|
|2597.39|
|4394.38|
|4358.04|
|4396.14|
| 1451.0|
| 733.11|
+-------+
only showing top 20 rows

+-------+
|  sales|
+-------+
|5205.27|
|5512.32|
|7737.93|
|7516.08|
|5404.62|
|7209.11|
|7329.06|
| 7374.1|
|10993.5|
|8014.82|
|5372.57|
|7290.36|
|9064.89|
| 6075.3|
|6463.23|
|6120.34|
|7680.64|
|8014.82|
|7136.19|
|10172.7|
+-------+
only showing top 20 rows

In [7]:
sales5000minus = rowwiseCollection.select("sales5000minus").toDF()
sales5000plus = rowwiseCollection.select("sales5000plus").toDF()
allsalesDF = sales5000plus.union(sales5000minus)
allsalesDF.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+---------+---------------+-------+---------------+--------+------+--------+-------+------------+----+-----------+--------+----------+
|ordernumber|quantityordered|priceeach|orderlinenumber|  sales|      orderdate|  status|qtr_id|month_id|year_id| productline|msrp|productcode|dealsize|customerid|
+-----------+---------------+---------+---------------+-------+---------------+--------+------+--------+-------+------------+----+-----------+--------+----------+
|      10159|             49|    100.0|             14|5205.27|10/10/2003 0:00| Shipped|     4|      10|   2003| Motorcycles|  95|   S10_1678|  Medium|         5|
|      10188|             48|    100.0|              1|5512.32|11/18/2003 0:00| Shipped|     4|      11|   2003| Motorcycles|  95|   S10_1678|  Medium|         8|
|      10341|             41|    100.0|              9|7737.93|11/24/2004 0:00| Shipped|     4|      11|   2004| Motorcycles|  95|   S10_1678|   Large|        21|
|      10417|         